In [4]:
from bs4 import BeautifulSoup
import urllib
import unicodedata

In [27]:
url = 'http://comediatheque.net/le-joker/'
# url = 'http://comediatheque.net/librairie-theatrale-en-ligne-achat-livre/'

raw_webpage = urllib.urlopen(url).read()

In [48]:

# webpage = raw_webpage.decode('utf8')

webpage = unicodedata.normalize("NFC", webpage)#.replace('&nbsp;', '')

#webpage = webpage.decode('unicode_escape').replace('\xa0',' ').encode('ascii','ignore')


#print webpage
soup = BeautifulSoup(webpage, 'html.parser')
print soup.prettify()

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="fr-FR" prefix="og: http://ogp.me/ns#" xmlns="http://www.w3.org/1999/xhtml">
 <head profile="http://gmpg.org/xfn/11">
  <meta content="; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <title>
   librairie théâtrale le joker comédies théâtre pièces télécharger textes gratuit
  </title>
  <meta content="WordPress 4.8.1" name="generator"/>
  <!-- leave this for stats -->
  <!-- <link rel="stylesheet" type="text/css" media="screen" href="http://comediatheque.net/wp-content/themes/techozoic-fluid/style.css" /> -->
  <link href="//comediatheque.net/wp-content/cache/wpfc-minified/e5b2d85a81df799824d78bc06184bf97/1502289821index.css" media="screen" rel="stylesheet" type="text/css"/>
  <!--[if IE]>
<style type="te

In [138]:
all_pTags = soup.find_all('p')
# print list_of_pTags
# print len(listOfAllpTags)

ind = 0

for i, s in enumerate(all_pTags):
    if 'www.sacd' in str(s):
        ind = i#all_pTags.index(i)


useful_pTags = all_pTags[ind+1:]
# print useful_pTags
# print type(useful_pTags[1])

# useful_pTags is a LIST of bs4.element.Tag objects

def parse_dialog(dialog, speakers):
    parts = dialog.split(' – ')
    
    raw_speaker = parts[0].split(' ')
    speaker = raw_speaker[0]
    
    utt = parts[1]
    
    if speaker in speakers:
        speaker = speakers[speaker]
    else:
        speakers[speaker] = len(speakers) + 1
    
    if speaker == '':
        print ' --> ', dialog
    
    return speaker, utt

dialogs = []
current_dialog = []
speakers = {}

for t in useful_pTags:
#     print len(t.contents)
    
    ut_s = ''
    for c in t.contents:
        ut_s += str(c.encode('utf8'))
    
    #detect dialog
    if ' – ' in ut_s:
        if t.contents and len(t.contents) > 1:
            for ut in t.contents:
                ut_s = str(ut.encode('utf8'))


            #check if child element is also a TAG like parent
            if type(ut) == type(t):

                if '<em>' in ut_s:
                    if '(soupirant)' in ut_s:
                        print ut_s
                    if ' – ' in ut_s :
                        if ut.contents and ut.contents > 1:
                            #remove embeded emphasis tag
                            ut_s = ''
                            for c in ut.contents:
                                ut_s += str(c.encode('utf8'))
    #                         ut_s = str(ut.contents[0].encode('utf8'))
    #                         print ' --> ', ut_s, ' | ', str(ut.contents[0].encode('utf8'))

                        current_dialog.append(parse_dialog(ut_s, speakers))
                    else:
    #                     print ut_s
                        dialogs.append(current_dialog)
                        current_dialog = []


            elif ' – ' in ut_s :
                if '(soupirant)' in ut_s:
                    print ut_s
                result = parse_dialog(ut_s, speakers)
                current_dialog.append(result)
            
        else:
            result = parse_dialog(ut_s, speakers)            
            current_dialog.append(result)
      
    
#             print ' --> ', ut
#             print 'BOOM'
    
print speakers

 -->   – Ouf… Heureusement que j’ai eu le bon réflexe…
{'': 3, 'P\xc3\xa8re': 5, 'Alex': 1, '<strong>Paris': 6, 'Joker': 4, '<strong>\xc2\xa9': 7, '<em>Fred': 2}


In [133]:
result = '<dialog>\n'

for dialog in dialogs:
    
    for i, utt in dialog:
        result += '\t<s>\n'
        
        t_result = '\t\t<utt uid="' + str(i) + '">' + utt + '</utt>\n'
        result += t_result
    
        result += '\t</s>\n'
    
    
result += '</dialog>'

print result

with open('result.xml', 'w') as f:
    f.write(result)
    
# text is a list of strings (no <p> or any HTML tags in these strings)

<dialog>
	<s>
		<utt uid="Alex">Bon sang… Quel crétin… </utt>
	</s>
	<s>
		<utt uid="1">Faudrait que j’arrête avec la télé… Si je ne bosse que pendant les pauses publicitaires, ça ne va pas avancer…</utt>
	</s>
	<s>
		<utt uid="1">Et merde… Si on me dérange tout le temps aussi, je ne vais jamais y arriver…</utt>
	</s>
	<s>
		<utt uid="1">Oui Fred, comment tu vas ma chérie?</utt>
	</s>
	<s>
		<utt uid="Fred">Bonjour Alex. Alors c’est toujours moi qui dois t’appeler… Qu’est-ce que tu fais?</utt>
	</s>
	<s>
		<utt uid="1">Ben tu vois, je suis vissé à mon bureau là, je bosse…</utt>
	</s>
	<s>
		<utt uid="2">Tu viens après? C’est pour toi que j’ai acheté ce grand lit chez Ikéa. Et je dors toute seule dedans. </utt>
	</s>
	<s>
		<utt uid="1">Écoute, ce soir, je crois que ça va être dur…</utt>
	</s>
	<s>
		<utt uid="2">Dur? Tu dis ça à chaque fois! Ça finit par être vexant… C’est si dur que çade passer la nuit avec moi?</utt>
	</s>
	<s>
		<utt uid="1">J’ai un scénario à terminer, là, et…</utt